# DMRG Test: Block Growth Version

Edited by [Song Menghan](https://quantummc.xyz/members/menghan-song/), Aug.2022.

This is a notebook for a simple traditional DMRG algorithm with block growth method. The code shown in this notebook is adapted from Garrison and Mishmash in the [simple-dmrg website](https://simple-dmrg.readthedocs.io/en/latest/).  

For further information, one can read the following two review articles by Malvezzi  [arXiv:cond-mat/0304375](https://arxiv.org/abs/cond-mat/0304375v1) and Schollwoeck [arXiv:1008.3477](https://arxiv.org/abs/1008.3477v2).

## 1. Introduction

### 1.1 Background
The core idea behind the DMRG is to find a way in which one can increase the system size without increasing the Hilbert space. 

This is typically done in two steps:

$\bullet $ Construct large systems from samll systems by gradually increase the system size. So far, the Hilbert space grows exponentially and we have not improved anything.

$\bullet $ Truncate the Hilbert space to a certain value, keeping its size invariant when more sites are added.

Here two key steps are in order:

$\bullet$ How to enlarge the system?

$\bullet$ What is the criterion to truncate the Hilbert space?


### 1.2 Procedure
To start with, we consider a single site which is described by the state $d_i (i=1,...,D)$. Define a block $B(l,m)$ consist $l$ sites and $H_B$ denotes the Hamiltonion of length $l$ chain <font color=red> [please explain explicitly what is $H_B$ (Heisenberg model?)]</span>. 

$m$ is the dimension of the basis in which we represent the operators of the block (If we do nothing, $m=D^{L}$). 

The block is then grown by adding a site to it, forming an enlarged block, $B^{e}$. Denote $\left | b_1  \right \rangle ...\left | b_m  \right \rangle $ and $\left | d_1  \right \rangle ...\left | d_D  \right \rangle $ the basis of the block and the additional site, the basis of the enlarged block is their direct product,

$$
\left | b_k^{e}  \right \rangle=\left | b_i  \right \rangle \otimes \left | d_j  \right \rangle,
$$
<font color=red> [please explain what are the meanings of super/subscript e, k, i and j ?]</span>

and thus the dimension of the Hilbert space for $B^{e}$ is the product of that for $B(l,m)$ and a site, $i.e. m\times D$.

The next step is to form the superblock which consists of two enlarged blocks connected to each other. 

<font color=red> [can we add a simple figure here with two blocks on each side and two dots in the center?]</span>
<br/><br/>

### 1.3 OBC of infinite chain
This sample code considers the open boundary conditions (OBC) and aims to compute the ground state energy of an infinite chain. 

For infinite chain algorithm, the superblock is formed by connecting the enlarged block $B_{e}$ and its replica(reflected) $B_{e}^{'}$ together. Compute the Hamiltonian of the superblock and diagonalize it to find its ground state energy and wavefunction,

$$
\left | \Psi_{gs} \right \rangle =\sum_{i=1}^{m\times D}\sum_{j=1}^{m\times D} a_{i,j}\left | b_{i}^{e} \right \rangle\otimes \left | b_j^{'e} \right \rangle,
$$

which is written in a basis that is the tensor product of the system(e.g. left enlarged block) and the environment(e.g. right replica). We can further compute the density matrix of the system,

$$
\rho _{i,i^{'}}=\sum_{j=1}^{m\times D}a_{i,j}a_{i^{'},j}^{*} .
$$

**In statistic mechanics, density matrix tells us which states of the system contribute the most to the above ground state**. 

The density matrix has the same dimension and block-diagonal structure of $H_e$, for the enlarged block. Let's denote 

$\left | u_{a} \right \rangle$ the eigenvectors 

and 

$w_{a}(a=1,...,m\times D)$ the eigenvalues of $\rho$, 

then $\sum_{a}w_a =1$ with $w_a$ is the probability of the system being in the state $\left | u_{a} \right \rangle$ when the system is at the ground state $\left | \Psi_{gs} \right \rangle$.
<br/><br/>

### 1.4 Reduced density matrix

The structure of the density matrix hints that one can keep only the most relevant states (with the largest few $w_{a}$) to describe the system at the ground states, which reduces the exponentially large Hilbert space into a fixed size. To make an optimal choice, we need to order the state $\left | u_{a} \right \rangle$ by their eigenvalues $w_{a}$ in a decending order and we only use the largest $m$ eigenvectors as a new reduced basis to describe $B_e$. Now the enlarged block is interpreted as $B_e (l+1,m)$ instead of $B_e (l+1,m\times D)$. 

Since we adapts a new basis for $B_e$. We need to rotate the operators into the new basis. For example, the Hamiltonian is transformed as,

$$
H_{B(l+1,m)}=OH_{e}O^{\dagger},
$$

where the rows of the $(m,(m\times D))$ matrix $O$ are the selected  $\left | u_{a} \right \rangle$. Again, we clearly see that the Hilbert space size is cut back to $m$. 

<font color=red> [can we add a simple figure here with the dimension of the arrays indicated?]
$$
\begin{pmatrix}
1 & 2 \\
4 & 5 
\end{pmatrix}=
\begin{pmatrix}
1 & 2 & 3\\
4 & 5 & 6
\end{pmatrix}
\begin{pmatrix}
4 & 5 & 6\\ 
7 & 8 & 9 \\
10 & 11 & 12 \\
\end{pmatrix}
\begin{pmatrix}
4 & 5 \\ 
7 & 8  \\
10 & 11 \\
\end{pmatrix}
$$

</span>

Then we continue add a site to the exsisting block and repeat the routine. We are able to reach a large system size since the Hilbert space size has a ceiling $m$. See whether the ground state energy of the superblock converges. If so, we successfully approximate the thermal dynamic limit.


## 2. Sample code

In [1]:
from __future__ import print_function, division  
import numpy as np
from scipy.sparse import kron, identity
from scipy.sparse.linalg import eigsh  # Lanczos routine from ARPACK
from collections import namedtuple

In [2]:
Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

model_d = 2 # single site basis for spin-1/2


In [3]:
def is_valid_block(block):
    for op in block.operator_dict.values():
        if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
            return False
    return True
is_valid_enlarged_block = is_valid_block

In [5]:
def GetSpinOp(spin):
    '''
    Given a spin quantum number
    return various spin operators and one site H
    '''
    Jp = lambda j,m: np.sqrt(j*(j+1)-m*(m+1))
    Jm = lambda j,m: np.sqrt(j*(j+1)-m*(m-1))
    
    Mdim = int(2*spin+1)
    
    Sz = np.zeros((Mdim,Mdim))
    Sp = np.zeros((Mdim,Mdim))
    Sm = np.zeros((Mdim,Mdim))
    
    for ii in range(Mdim):
        sm = spin-ii
        Sz[ii,ii]= sm
        if ii > 0:
            Sp[ii-1,ii]=Jp(spin,sm)
        if ii < 2*spin:
            Sm[ii+1,ii]=Jm(spin,sm)
    Sx = (Sp+Sm)/2
    Sy = (Sp-Sm)/(2j)
    
    H1 = np.zeros((Mdim,Mdim))
    
    return Sz,Sp,H1

In [6]:
Sz1,Sp1,H1 = GetSpinOp(1/2)
Sz1,Sp1,H1

(array([[ 0.5,  0. ],
        [ 0. , -0.5]]), array([[0., 1.],
        [0., 0.]]), array([[0., 0.],
        [0., 0.]]))

Adding a site to the block, compute the enlarged H as follows:
$$
H_e=H_B\otimes I_d+\frac{J}{2}(S_b^{+}\otimes S_d^{-}+S_b^{-}\otimes S_d^{+})+J_z S_b^{z}\otimes S_d^{z},
$$

considering the internal block part and the interaction part with a new spin.

In [7]:
def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of the enlarged H
    """
    Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    J = Jz = 1.
    TwoSiteH = (J/2)*(kron(Sp1,Sp2.conjugate().transpose())+kron(Sp1.conjugate().transpose(),Sp2))\
    +Jz*kron(Sz1, Sz2)
    
    return TwoSiteH


In [8]:
initial_block = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "conn_Sz": Sz1,
    "conn_Sp": Sp1,})

# conn refers to the connection operator, that is, the operator on the edge of
# the block, on the interior of the chain.  We need to be able to represent S^z
# and S^+ on that site in the current basis in order to grow the chain.

In [9]:
initial_block

Block(length=1, basis_size=2, operator_dict={'H': array([[0., 0.],
       [0., 0.]]), 'conn_Sz': array([[ 0.5,  0. ],
       [ 0. , -0.5]]), 'conn_Sp': array([[0., 1.],
       [0., 0.]])})

Together with the Block Hamiltonian $H_b$, we need to present the spin operators of the rightmost site of the $\mathbf{enlarged}$ block in the basis of the enlarged block (basis with one more site added). For example, the spin-1/2 $S_z$ transforms as:
$$
(S_r^{z})_e = I_b\otimes S_d^{z}=I_b\otimes \begin{bmatrix}
 0.5 & 0 \\
 0 & -0.5
\end{bmatrix}
$$
, where $I_b$ is the identity matrix of the block Hilbert space, $S_d^{z}$ is the $S_z$ operator for the righmost site.

In [10]:
def enlarge_block(block):
    """
    Enlarges the block by adding one site
    Return the Enlarged block
    """
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  Our basis becomes a
    # Kronecker product of the Block basis and the single-site basis.  NOTE:
    # `kron` uses the tensor product convention making blocks of the second
    # array scaled by the first.  As such, we adopt this convention for
    # Kronecker products throughout the code.
    enlarged_operator_dict = {
        "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1),
        "conn_Sz": kron(identity(mblock), Sz1),
        "conn_Sp": kron(identity(mblock), Sp1),
    }

    return EnlargedBlock(length=(block.length + 1),
                         basis_size=(block.basis_size * model_d),
                         operator_dict=enlarged_operator_dict)

In [11]:
def ChangeBasis(operator,trans_mat):
    '''
    Transform the operator into the truncated basis with trans_mat
    '''
    op_new = trans_mat.conjugate().transpose().dot(operator.dot(trans_mat))
    return op_new
    

Here we define a single DMRG step with one site added. 

Note that the formation of the superblock Hamiltonian is similar to that for $H_e$. They both consist three parts: Hamiltonian for both enlarged block (in the construction for $H_e$, one can regard the additional site as a right block with a single site Hamiltonion which is zero) and the interaction between the connecting sites.

$$
H_s=H_e\otimes I_e^{'}+I_e \otimes H_e^{'}+\frac{J}{2}((S_r^{+})_e \otimes (S_r^{-})_{e}^{'}+(S_r^{-})_e\otimes (S_r^{+})_e^{'})+J_z (S_r^{z})_e\otimes (S_r^{z})_e^{'},
$$

where the subscript $r$ means the rightmost site of the block, and $'$ stands for the replica enlarged block.

We sum up the eigenvalues of the discarded eigenstates $(1-\sum_{a=1}^{m}w_a)$ as the measure for the truncation error. In many cases, this number is roughly proportional to the error in the energy. 

In [13]:
def single_dmrg_step(sys,env,mm):
    '''
    single DMRG step with the maximum truncation mm, mm states in the new basis
    '''
    assert is_valid_block(sys)
    assert is_valid_block(env) #check whether the block is legal
    
    sys_enl = enlarge_block(sys)
    
    if sys is env: #if they are same, not need to compute again
        env_enl = sys_enl
    else:
        env_enl = enlarge_block(env)
    
    assert is_valid_enlarged_block(sys_enl)
    assert is_valid_enlarged_block(env_enl) #check whether enl_block is legal
    
    #construct the superblock Hamiltonian.
    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.operator_dict
    env_enl_op = env_enl.operator_dict
    superblock_H = kron(sys_enl_op["H"],identity(m_env_enl))\
                  +kron(identity(m_sys_enl), env_enl_op["H"])\
                  +H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"], env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])
    
    #find the superblock ground state. 
    #'SA' means find the 'smallest in amplitude' eig_value
    #|psi0> = sigma{i}sigma{j}: a(i,j)|bi>|bj>
    #|bi>, |bj> are the basis of the sys and env
    #reshape the eigenvector such that (row,colum) corresponds to (sys,env)
    #Now, the matrix elements of psi0 are a(i,j)
    #rho is the density matrix
    (energy,), psi0 = eigsh(superblock_H, k=1, which="SA")
    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0,psi0.conjugate().transpose())
    
    evals,evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eig_val, eig_vec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eig_val, eig_vec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    #Based on the truncation 'mm', compute trans_mat
    my_m = min(len(possible_eigenstates),mm)
    trans_mat = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i,(eig_val,eig_vec) in enumerate(possible_eigenstates[:my_m]):
        trans_mat[:,i] = eig_vec
    
    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    print("truncation error:", truncation_error)
    
    #change the basis for each operator
    new_operator_dict={}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = ChangeBasis(op,trans_mat)
    
    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)
    
    return newblock, energy
    
    
    
    

### 2.1 Infinite system algorithm

In [14]:
def infinite_system(L,mm):
    block = initial_block
    #keep growing the block
    #let the right block = left block, i.e. env=sys
    while 2*block.length<L:
        #1st step, grow 1 to 2 sites, hence 4 sites in superblock
        print('L =',block.length*2 + 2) 
        block, energy = single_dmrg_step(block, block, mm)
        print('E/L =', energy/(block.length*2))


In [15]:
def Run_infinite(L):
    if __name__ == "__main__":
        np.set_printoptions(precision=10, suppress=True, threshold=10000, linewidth=300)
        infinite_system(L, mm=20)
        
        

In [16]:
Run_infinite(L=100)

L = 4
truncation error: -4.440892098500626e-16
E/L = -0.4040063509461094
L = 6
truncation error: 3.3306690738754696e-16
E/L = -0.41559618898132095
L = 8
truncation error: 2.220446049250313e-16
E/L = -0.42186657483598666
L = 10
truncation error: 9.166301051521941e-11
E/L = -0.42580352072828853
L = 12
truncation error: 3.1294844493601204e-10
E/L = -0.4285075524242812
L = 14
truncation error: 1.6584675854858233e-09
E/L = -0.4304803311837219
L = 16
truncation error: 4.333193359684628e-09
E/L = -0.43198356451228404
L = 18
truncation error: 1.0037189857037276e-08
E/L = -0.4331672644472634
L = 20
truncation error: 1.8536259638324282e-08
E/L = -0.4341236316813933
L = 22
truncation error: 3.338432952304515e-08
E/L = -0.43491247896381285
L = 24
truncation error: 4.786561780267817e-08
E/L = -0.4355743093654312
L = 26
truncation error: 7.762831222279942e-08
E/L = -0.436137536172096
L = 28
truncation error: 9.409576695595945e-08
E/L = -0.4366226762361586
L = 30
truncation error: 1.4473940113113315e

### 2.2 Finite system algorithm

In this fintie system algorithm, we aims to simulate the finite chain with length L precisely instead of reaching the thermal dynamic limit. Therefore, we need to grow the block to our required length, fix it and we do sweeps to equilibrate the chain. The overall procedure is listed below:
1. First use the infinite size algorithm until the superblock size reaches the chain size L under investigation. After every truncation save all operators of the reduced block to disk.
2. Denote the length of the system block by $l$. Load the environment block of size $L − l − 2$ from disk, this is the right block.
3. Call single_dmrg_step with inputs sys and env which are save in the disk. Then, the system block's length increases by one, $l\to l+1$.
4. Continue step 2 and 3 untill the environment block becomes a single site.
5. Then reverse the system and the environment, do step 4 untill energy converges.


In [22]:
def graphic(sys_block, env_block, sys_label="l"):
    """This is a graphic representation of the block,
    =: stands for the system
    -: stands for the environment
    **: stands for the intermedium two sites
    """
    assert sys_label in ("l", "r")
    graphic = ("=" * sys_block.length) + "**" + ("-" * env_block.length)
    if sys_label == "r":
        # The system should be on the right and the environment should be on
        # the left, so reverse the graphic.
        graphic = graphic[::-1]
    return graphic

In [23]:
def finite_system_algorithm(L, m_warmup, m_sweep_list):
    '''
    This function print the process of sweeps and return the ground state energy of the finite system.
    A sweep goes as:
    ====**---- >> ======**-- >> -------**= >> --**====== >> =**------- >> ====**----
    
    L: system length
    m_warmup: the truncation length used for growing the system to length L
    m_sweep_list: a list contains different 'mm' for consideration.
    '''
    assert L % 2 == 0  # require that L is an even number

    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = {}  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # ("l") and right ("r") block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    block = initial_block
    block_disk["l", block.length] = block
    block_disk["r", block.length] = block
    while 2 * block.length < L:
        # Perform a single DMRG step and save the new Block to "disk"
        # grow block size to L/2 and now the system is L/2 -2 
        print(graphic(block, block))
        block, energy = single_dmrg_step(block, block, mm=m_warmup)
        print("E/L =", energy / (block.length * 2))
        block_disk["l", block.length] = block
        block_disk["r", block.length] = block

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = "l", "r"
    sys_block = block; del block  # rename the variable
    for mm in m_sweep_list:
        while True:
            # Load the appropriate environment block from "disk"
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                # We've come to the end of the chain, so we reverse course.
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

            # Perform a single DMRG step.
            print(graphic(sys_block, env_block, sys_label))
            sys_block, energy = single_dmrg_step(sys_block, env_block, mm=mm)

            print("E/L =", energy / L)

            # Save the block from this step to disk.
            block_disk[sys_label, sys_block.length] = sys_block

            # Check whether we just completed a full sweep.
            if sys_label == "l" and 2 * sys_block.length == L:
                break  # escape from the "while True" loop

In [24]:
def Run_finite(L):
    if __name__ == "__main__":
        np.set_printoptions(precision=10, suppress=True, threshold=10000, linewidth=300)

        finite_system_algorithm(L, m_warmup=10, m_sweep_list=[20])

In [25]:
Run_finite(10)

=**-
truncation error: -4.440892098500626e-16
E/L = -1.161437827766148
==**--
truncation error: 0.0009637667328763477
E/L = -1.2283791615707664
===**---
truncation error: 0.00023686234322362054
E/L = -1.2638894710278605
====**----
truncation error: 0.0022339429937699995
E/L = -1.287443479918379
=====**---
truncation error: 4.7292809124410695e-07
E/L = -1.2867436738537357
======**--
truncation error: 3.42379409379312e-07
E/L = -1.2874997913701742
-------**=
truncation error: 9.992007221626409e-16
E/L = -1.2874994815824075
------**==
truncation error: 3.4237941115566883e-07
E/L = -1.2874997913701731
-----**===
truncation error: 9.953071220536458e-07
E/L = -1.287500752718821
----**====
truncation error: 1.1630179393984363e-06
E/L = -1.2884333373528656
---**=====
truncation error: 1.1027216428871611e-06
E/L = -1.2886570592055053
--**======
truncation error: 4.471280863338478e-07
E/L = -1.2894441655187454
=**-------
truncation error: 3.3306690738754696e-16
E/L = -1.2894437736826516
==**----

## 3. Benchmark with ED

In [26]:
def ED_1d(N,OBC=None):
    dimension=2**N
    #spin chain with all spin-down
    z='0'*N
    # initialize hamiltonian
    H=np.zeros((dimension,dimension))
    # Matrix Construction
    for a in range(dimension):
        if OBC==True:
            num=N-1
        else:
            num=N
        for i in range(num): #N-1 if OBC

            j=np.mod(i+1,N)
            state_chain=bin(a)[2:] # the first two should be omitted for this 'bin' function
            l=len(state_chain)
    #        print(state_chain)
            state_chain=z[0:N-l]+state_chain # make the length equal to N
            if state_chain[i]==state_chain[j]: # i=j only diagonal elements
                H[a,a]+=0.25
            else:                              # else, the raising/lowering operators also have contributions
                H[a,a]-=0.25
                # then exchange i,j
                element_i=state_chain[i]
                element_j=state_chain[j]
                #flip
                if j==0:  #here we are doing the concatenation of string (you can try other methods)
                  state_chain1=element_i+state_chain[1:N-1]+element_j
                else:
                  state_chain1=state_chain[0:i]+element_j+element_i+state_chain[j+1:]
                b=int(state_chain1,2)
                H[a,b]+=0.5
    eig_value=np.real(np.linalg.eig(H)[0])# eigen_values
    eig_vec=np.real(np.linalg.eig(H)[1])  # eigenstates
    
    idx_sorted1 = np.argsort(eig_value)
    eig_value=eig_value[idx_sorted1]
    eig_vec=eig_vec[:,idx_sorted1]
    eig_value[0]/N, eig_vec[:,0]
    energy_level=np.sort(eig_value/N)
    #return Ground Energy per site
    return energy_level[0]
    

In [27]:
for L in [4,6,8,10]:
    print('L =',L)
    E0 = ED_1d(L,OBC=True)
    print('E/L =', E0)

L = 4
E/L = -0.40400635094610937
L = 6
E/L = -0.4155961889813223
L = 8
E/L = -0.42186657483598755
L = 10
E/L = -0.4258035207282885


## 4. Test for spin-1 chain

In [28]:
Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

model_d = 3 # single site basis for spin-1/2

Sz1,Sp1,H1 = GetSpinOp(1)
Sz1,Sp1,H1


(array([[ 1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0., -1.]]), array([[0.          , 1.4142135624, 0.          ],
        [0.          , 0.          , 1.4142135624],
        [0.          , 0.          , 0.          ]]), array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

In [29]:
initial_block = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "conn_Sz": Sz1,
    "conn_Sp": Sp1,})

In [30]:
Run_finite(6)

=**-
truncation error: -2.220446049250313e-16
E/L = -1.1614378277661481
==**--
truncation error: 0.0009637667328746824
E/L = -1.2283791615707684
---**=
truncation error: -2.220446049250313e-16
E/L = -1.2276091976687162
--**==
truncation error: 1.120216479577607e-07
E/L = -1.2283791615707678
=**---
truncation error: 1.1102230246251565e-16
E/L = -1.2283789756826693
==**--
truncation error: 1.1202164706958229e-07
E/L = -1.2283791615707689


In [31]:
Run_infinite(50)

L = 4
truncation error: 4.440892098500626e-16
E/L = -1.1614378277661475
L = 6
truncation error: 1.1202164684753768e-07
E/L = -1.2283791615707684
L = 8
truncation error: 1.6147017241063821e-06
E/L = -1.265578933719902
L = 10
truncation error: 3.588730170811516e-06
E/L = -1.289449145847944
L = 12
truncation error: 7.155731815089439e-06
E/L = -1.306148675593042
L = 14
truncation error: 1.1752284542088098e-05
E/L = -1.3185250883940771
L = 16
truncation error: 1.3970248320949175e-05
E/L = -1.3280824162835736
L = 18
truncation error: 2.084283886683025e-05
E/L = -1.3356938899770974
L = 20
truncation error: 2.011019233982836e-05
E/L = -1.3418995850937752
L = 22
truncation error: 2.77606064185143e-05
E/L = -1.3470580557683858
L = 24
truncation error: 2.5018305201518487e-05
E/L = -1.3514098883352534
L = 26
truncation error: 3.2103085188972535e-05
E/L = -1.3551314950194588
L = 28
truncation error: 2.8674917264881472e-05
E/L = -1.3583462897261112
L = 30
truncation error: 3.4519641600283535e-05
E/L